In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from imutils import paths
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from keras import applications

Using TensorFlow backend.


In [3]:
test_dir = "/content/drive/My Drive/ambulance_prediction/data/test"
modelPath = "/content/drive/My Drive/ambulance_prediction"
img_width, img_height = 150, 150
batch_size = 16
test_size = len(list(paths.list_images(test_dir)))

In [ ]:
'''
The below 4 code cells are used for using pretrained vgg16 model
'''
datagen = ImageDataGenerator(rescale=1. / 255)
# build the VGG16 network
base_model = applications.VGG16(include_top=False, weights='imagenet')
generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
predict_size_test = int(math.ceil(test_size / batch_size))
bottleneck_features_test = base_model.predict_generator(generator, predict_size_test)
test_labels = np.array([0] * (test_size // 2) + [1] * (test_size // 2))

# load the model we saved
model = load_model(os.path.join(modelPath,'first_try_pretrained.h5'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

score = model.predict_classes(bottleneck_features_test).reshape((1,74))
print(score)
print(len(bottleneck_features_test))
print(test_labels)

In [ ]:
batch_holder = np.zeros((test_size, img_width, img_width, 3))
# predicting images
for i,img1 in enumerate(paths.list_images(test_dir)):
  img = image.load_img(os.path.join(test_dir, img1), target_size=(img_width, img_height))
  batch_holder[i:] = img

In [ ]:
result_1 = {'ambulance': 0, 'notambulance': 1}
key_list = list(result_1.keys())
val_list = list(result_1.values())

plotx1 = test_size//3 if test_size%3 == 0 else (test_size//3)+1

In [ ]:
fig = plt.figure(figsize=(15,80))
for i,img in enumerate(batch_holder):
  fig.subplots_adjust(wspace=0.2, hspace=0.2)
  fig.add_subplot(plotx1,3,i+1)
  plt.title(key_list[val_list.index(score[0][i])])
  plt.imshow(img/256.)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
'''
The below 4 method is for normal cnn method
'''
batch_holder = np.zeros((test_size, img_width, img_width, 3))
# dimensions of our images
#img_width, img_height = IMG_SIZE, IMG_SIZE

# load the model we saved
model = load_model(os.path.join(modelPath,'first_try_pretrained_chk.h5'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# predicting images
for i,img1 in enumerate(paths.list_images(test_dir)):
  img = image.load_img(os.path.join(test_dir, img1), target_size=(img_width, img_height))
  batch_holder[i:] = img
  #x = image.img_to_array(img)
  #x = np.expand_dims(x, axis=0)
  #images = np.vstack([x])

classes = model.predict_classes(batch_holder)
print(classes.shape)

In [ ]:
result_1 = {'ambulance': 0, 'notambulance': 1}
key_list = list(result_1.keys())
val_list = list(result_1.values())

plotx1 = batchsize//3 if batchsize%3 == 0 else (batchsize//3)+1

In [ ]:
fig = plt.figure(figsize=(15,75))
for i,img in enumerate(batch_holder):
  fig.subplots_adjust(wspace=0.2, hspace=0.2)
  fig.add_subplot(plotx1,3,i+1)
  plt.title(key_list[val_list.index(classes[i][0])])
  plt.imshow(img/256.)
plt.show()

Output hidden; open in https://colab.research.google.com to view.